In [ ]:
!pip install ace_tools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = "/content/drive/My Drive/Colab Notebooks/mortgage_covenant_data.csv"
df = pd.read_csv(file_path)

df.info()
df.head()


NameError: name 'pd' is not defined

In [ ]:
# Check missing values in the dataset

missing_values = df.isnull().sum()

# Convert to DataFrame for better readability

missing_df = missing_values.to_frame(name="Missing Count")

# Display the missing values summary

print(missing_df)


In [ ]:
# Dropping rows with missing census tract data

df_cleaned = df.dropna(subset=["census_tract"])

In [ ]:
# Filling missing property values with median

df_cleaned["property_value"] = df_cleaned.groupby("census_tract")["property_value"].transform(
    lambda x: x.fillna(x.median())
)

In [ ]:
# Filling missing income values

df_cleaned["income"] = df_cleaned.groupby("census_tract")["income"].transform(
    lambda x: x.fillna(x.median())
)

In [ ]:
# Filling missing interest rates with the median interest rate

df_cleaned["interest_rate"] = df_cleaned.groupby("loan_amount")["interest_rate"].transform(
    lambda x: x.fillna(x.median())
)


In [ ]:
# Replacing missing covenant density with 0

df_cleaned["covenant_density"] = df_cleaned["covenant_density"].fillna(0)

In [ ]:
# Verifing that missing values have been handled

missing_values_after = df_cleaned.isnull().sum()

In [ ]:
# Remove Ramsey County data from the dataset

ramsey_county_tracts = df_cleaned[df_cleaned["census_tract"].astype(str).str.startswith("2.7")]

In [ ]:
# Drop Ramsey County records

df_cleaned = df_cleaned[~df_cleaned["census_tract"].isin(ramsey_county_tracts["census_tract"])]

In [ ]:
# Confirm dataset size after removing Ramsey County

df_cleaned.shape


In [ ]:
# Mortgage Approval Rates by Race

approval_rates = df_cleaned.groupby("derived_race")["was_approved"].mean().sort_values()

In [ ]:
# Property Values by Covenant Presence

covenant_vs_property_value = df_cleaned.groupby("covenant_count")["property_value"].mean()

In [ ]:
# Interest Rates by Race

interest_rates_by_race = df_cleaned.groupby("derived_race")["interest_rate"].mean()

In [ ]:
# Display summary statistics in a table format

summary_df = pd.DataFrame({
    "Approval Rate": approval_rates,
    "Average Interest Rate": interest_rates_by_race,
}).fillna("-")

In [ ]:
from IPython.display import display

In [ ]:
# Display the summary DataFrame

display(summary_df)

In [ ]:
import scipy.stats as stats

In [ ]:
# ANOVA Test for Interest Rate Differences by Race

race_groups = [group.dropna().values for _, group in df_cleaned.groupby("derived_race")["interest_rate"]]

In [ ]:
# Run ANOVA test

anova_result = stats.f_oneway(*race_groups)

In [ ]:
# Chi-Square Test for Mortgage Approvals by Race

approval_counts = df_cleaned.groupby(["derived_race", "was_approved"]).size().unstack()
chi2_result = stats.chi2_contingency(approval_counts.fillna(0))

In [ ]:
# Display results

stat_results = pd.DataFrame({
    "Test": ["ANOVA - Interest Rate by Race", "Chi-Square - Loan Approvals by Race"],
    "Statistic": [anova_result.statistic, chi2_result.statistic],
    "P-Value": [anova_result.pvalue, chi2_result.pvalue]
})

In [ ]:
from IPython.display import display

# Display the statistical test results

display(stat_results)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Bar Chart: Mortgage Approval Rates by Race

plt.figure(figsize=(10, 5))
approval_rates.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Mortgage Approval Rates by Race", fontsize=14)
plt.xlabel("Race", fontsize=12)
plt.ylabel("Approval Rate", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
# Box Plot: Interest Rates by Race

plt.figure(figsize=(12, 6))
sns.boxplot(x="derived_race", y="interest_rate", data=df_cleaned, palette="pastel")
plt.title("Distribution of Interest Rates by Race", fontsize=14)
plt.xlabel("Race", fontsize=12)
plt.ylabel("Interest Rate (%)", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
# Scatter Plot: Historical Covenants vs. Property Values

plt.figure(figsize=(10, 6))
sns.scatterplot(x=df_cleaned["covenant_count"], y=df_cleaned["property_value"], alpha=0.5)
plt.title("Relationship Between Covenant Count and Property Values", fontsize=14)
plt.xlabel("Covenant Count", fontsize=12)
plt.ylabel("Property Value ($)", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()
